## CLASSIFYING NAMES WITH A CHARACTER-LEVEL RNN

https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html    

### Data downloading

In [1]:
!ls

0-PyTorch-Tutorial.ipynb	 3-Basic-RNN.ipynb		      LICENSE
1-Learning-Pytorch-60mins.ipynb  3-Name-categorisation-PyTorch.ipynb
2-Torch-With-Examples.ipynb	 Data


In [2]:
# downloading 
! cd ./Data ; wget https://download.pytorch.org/tutorial/data.zip

--2019-08-23 09:30:47--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.249.210.17, 13.249.210.76, 13.249.210.112, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.249.210.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  1.91MB/s    in 1.4s    

2019-08-23 09:30:50 (1.91 MB/s) - ‘data.zip’ saved [2882130/2882130]



In [3]:
# unzipping
! cd ./Data ; unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/Italian.txt  
  inflating: data/names/Japanese.txt  
  inflating: data/names/Korean.txt   
  inflating: data/names/Polish.txt   
  inflating: data/names/Portuguese.txt  
  inflating: data/names/Russian.txt  
  inflating: data/names/Scottish.txt  
  inflating: data/names/Spanish.txt  
  inflating: data/names/Vietnamese.txt  


In [4]:
!ls ./Data

data  data.zip	shakespeare.txt


In [5]:
!rm ./Data/data.zip ; mv ./Data/data ./Data/lang-names-data

In [6]:
!ls ./Data

lang-names-data  shakespeare.txt


In [7]:
!ls ./Data/lang-names-data/

eng-fra.txt  names


### Data Pre-processing

In [8]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

In [11]:
from pprint import pprint

In [9]:
def findFiles(path): 
    return glob.glob(path)

In [13]:
pprint(findFiles('./Data/lang-names-data/names/*.txt'))

['./Data/lang-names-data/names/Arabic.txt',
 './Data/lang-names-data/names/Chinese.txt',
 './Data/lang-names-data/names/Czech.txt',
 './Data/lang-names-data/names/Dutch.txt',
 './Data/lang-names-data/names/English.txt',
 './Data/lang-names-data/names/French.txt',
 './Data/lang-names-data/names/German.txt',
 './Data/lang-names-data/names/Greek.txt',
 './Data/lang-names-data/names/Irish.txt',
 './Data/lang-names-data/names/Italian.txt',
 './Data/lang-names-data/names/Japanese.txt',
 './Data/lang-names-data/names/Korean.txt',
 './Data/lang-names-data/names/Polish.txt',
 './Data/lang-names-data/names/Portuguese.txt',
 './Data/lang-names-data/names/Russian.txt',
 './Data/lang-names-data/names/Scottish.txt',
 './Data/lang-names-data/names/Spanish.txt',
 './Data/lang-names-data/names/Vietnamese.txt']


In [14]:
import unicodedata
import string

In [15]:
all_letters = string.ascii_letters + " .,;'"
all_letters

"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'"

In [16]:
n_letters = len(all_letters)
n_letters

57

In [17]:
# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [18]:
print(unicodeToAscii('Ślusàrski'))

Slusarski


In [21]:
# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('./Data/lang-names-data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

In [22]:
n_categories = len(all_categories)
n_categories

18

In [23]:
all_categories

['Arabic',
 'Chinese',
 'Czech',
 'Dutch',
 'English',
 'French',
 'German',
 'Greek',
 'Irish',
 'Italian',
 'Japanese',
 'Korean',
 'Polish',
 'Portuguese',
 'Russian',
 'Scottish',
 'Spanish',
 'Vietnamese']

In [24]:
import torch

In [25]:
# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for i, letter in enumerate(line):
        tensor[i][0][letterToIndex(letter)] = 1
    return tensor

In [29]:
print(letterToTensor('I'))


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])


In [30]:
print(letterToTensor('J'))


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])


In [31]:
print(lineToTensor('Jones').size())

torch.Size([5, 1, 57])


In [32]:
import torch.nn as nn

In [33]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [34]:
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

In [35]:
# One-hot vector for category
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

In [36]:
# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor

In [37]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item() / input_line_tensor.size(0)

In [38]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [39]:
rnn = RNN(n_letters, 128, n_letters)

n_iters = 100000
print_every = 5000
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

TypeError: forward() takes 3 positional arguments but 4 were given